In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tqdm import trange

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
disaster_info = pd.read_excel('./data/emdat_public_2023_05_31_query_uid-ldmOwE.xlsx', skiprows=6)

/Users/laurensniesten/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Data cleaning.
disaster_info['Start Date'] = pd.to_datetime(dict(year=disaster_info['Start Year'], month=disaster_info['Start Month'], day=disaster_info['Start Day']))
disaster_info['End Date'] = pd.to_datetime(dict(year=disaster_info['End Year'], month=disaster_info['End Month'], day=disaster_info['End Day']))
disaster_info = disaster_info.drop(columns=['Seq', 'OFDA Response', 'Local Time', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', "Reconstruction Costs ('000 US$)", "Insured Damages ('000 US$)", "Total Damages ('000 US$)", 'CPI', 'Adm Level', 'Admin1 Code', 'Admin2 Code'])

In [4]:
# Omit all natural disasters that took place on dates we do not have news articles on. 
disaster_info = disaster_info[disaster_info['Start Date'] < '2020-04-02'].copy()

In [5]:
# Create a list of the locations (keywords) involved in the natural disaster.
disaster_info['Location List'] = disaster_info['Location'].apply(lambda x: x.replace(' (', '_').replace('), ', '_').replace(', ', '_').replace(')', '').replace('; ', '_').split('_') if type(x) == str else [])

In [6]:
max_total_deaths = disaster_info.loc[disaster_info['Total Deaths'].idxmax()]['Total Deaths']
min_total_deaths = disaster_info.loc[disaster_info['Total Deaths'].idxmin()]['Total Deaths']
max_total_damages_adjusted = disaster_info.loc[disaster_info["Total Damages, Adjusted ('000 US$)"].idxmax()]["Total Damages, Adjusted ('000 US$)"]
min_total_damages_adjusted = disaster_info.loc[disaster_info["Total Damages, Adjusted ('000 US$)"].idxmin()]["Total Damages, Adjusted ('000 US$)"]

print(f'Maximum Total Deaths: {max_total_deaths}')
print(f'Minimum Total Deaths: {min_total_deaths}')
print(f'Maximum Total Damages Adjusted: {max_total_damages_adjusted}')
print(f'Minimum Total Damages Adjusted: {min_total_damages_adjusted}')

Maximum Total Deaths: 4140.0
Minimum Total Deaths: 1.0
Maximum Total Damages Adjusted: 113423011.0
Minimum Total Damages Adjusted: 42.0


In [7]:
news_data = pd.read_csv('./data/all-the-news-2-1.csv') 

In [8]:
news_data['Date'] = pd.to_datetime(dict(year=news_data['year'], month=news_data['month'], day=news_data['day']))
news_data['Time'] = news_data.date.str[-8:]

In [53]:
news_data.drop(columns=['date', 'year', 'month', 'day'], inplace=True)
news_data.rename(columns={'author': 'Author', 'title':'Title', 'url':'Url', 'section':'Section', 'publication':'Publication', 'article': 'Article'}, inplace=True)

KeyError: "['date', 'year', 'month', 'day'] not found in axis"

In [54]:
news_data['Article'].isna().sum()

104713

In [55]:
news_data = news_data[news_data['Article'].notna()].copy()

In [56]:
news_data['Article'] = news_data['Article'].apply(lambda x: x.replace("\'", "").replace("\xa0", " ").replace("  ", " ").replace("\n", " ") if type(x) == str else float("nan"))

In [12]:
disaster_info['mentions_1'] = 0
disaster_info['mentions_2'] = 0
disaster_info['mentions_3'] = 0

## Data Merging

In [14]:
pre_margin, post_margin = 1, 7

In [15]:
conv_dict = conv_dict = {'Cyclone Roanu': [['cyclone', 'roanu']],
             'Cyclone Winston': [['cyclone', 'winston']], 
             'Fort McMurray wildfires': [['fort', 'wildfires'],['mcmurray', 'wildfires']], 
             'Typhoon Butchoy (Nepartak)': [['typhoon','butchoy'],['typhoon', 'nepartak']], 
             'Tropical cyclone Zena': [['cyclone', 'Zena']], 
             'Yarloop busfires': [['yarloop','busfires']], 
             "Tropical storm 'Earl'": [['storm','earl']], 
             'Cholera': [['cholera']], 
             'Hurricane Newton': [['hurricane', 'newton']], 
             'Typhoon Megi': [['typhoon', 'megi']], 
             "Hurricane 'Matthew'": [['hurricane', 'matthew']], 
             'Typhoon Ferdie (Meranti)': [['typhoon', 'ferdie'], ['typhoon', 'meranti']], 
             'Tropical storm Otto': [['storm', 'otto']], 
             'Storm Jonas (Snowzilla)': [['storm','jonas'], ['storm', 'snowzilla']], 
             'Cyclone Ulla': [['cyclone', 'ulla']], 
             'Sand Fire': [['sand', 'fire']], 
             'Clayton Fire': [['clayton', 'fire']], 
             'Storm Mirinae': [['storm', 'mirinae']], 
             'Dengue': [['dengue']], 
             'Yellow fever': [['yellow', 'fever']], 
             'Typhoon Karen (Sarika)': [['typhoon', 'karen'], ['typhoon', 'sarika']], 
             'Typhoon Lawin (Haima)': [['typhoon' ,'lawin'], ['typhoon', 'haima']], 
             'Tropical storm Dianmu': [['storm','dianmu']], 
             'Typhoon Carina (Nida)': [['typhoon','carina'], ['typhoon','nada']], 
             "Tropical depression'16/Hurricane 'Nate'": [['depression', 'nate'], ['hurricane', 'nate']], 
             "Hurricane 'Maria'": [['hurricane','Maria']], 
             'Sir Ivan fire': [['fire','Ivan']], 
             'Gustafsen, Wildwood, Elephant Hill Fires': [['fire','Gustafson'], ['fire','wildwood'], ['fire','elephant', 'hill']], 
             'Debbie': [['Debbie']], 
             "Typhoon 'Hato'": [['typhoon','hate']], 
             "Hurricane 'Irma'": [['hurricane','irma']], 
             'Cyclone Mora': [['cyclone','mora']], 
             'Diphteria': [['diphtheria']], 
             'Tropical Storm Talas': [['storm','talas']], 
             'Typhoon Nesat & Haitang': [['typhoon','nesat'], ['typhoon','haitang']], 
             "Tropical storm 'Pakhar'/'Jolina'": [['storm','pakhar'], ['storm','jolina']], 
             "'Zeus'": [['zeus']], 'Eleanor': [['eleanor']], 
             "Tropical storm 'Alberto'": [['storm','alberto']], 
             'Volcan de Fuego': [['volcan','fuego']], 
             "Typhoon 'Maria' (Gardo)": [['typhoon', 'maria'], ['typhoon', 'gardo']], 
             "Typhoon 'Bebinca'": [['typhoon', 'bebinca']], 
             'Tropical Storm Yagi': [['storm', 'yagi']], 
             'Tropical storm Rumbia': [['storm', 'rumbia']], 
             "Cyclone 'Sagar'": [['cyclone', 'sagar']], 
             "Cyclone 'Josie'": [['cyclone', 'josie']], 
             "'Friederike' (David)": [['friederike'], ['david']], 
             'Influenza': [['influenza']], 
             'Hepatitis E': [['hepatitis e']], 
             "Tropical storm 'Ewiniar'": [['storm', 'ewiniar']], 
             "Tropical storm 'Ampil' (Inday)": [['storm', 'ampil'], ['storm', 'inday']], 
             "Typhoon 'Soulik'": [['typhoon', 'soulik']], 
             'Typhoon Mangkut (Ompong)': [['typhoon', 'mangkut'], ['typhoon', 'ompong']], 
             'Hurricane Michael': [['hurricane', 'michael']], 
             "Hurricane 'Beryl'": [['hurricane', 'beryl']], 
             "Tropical cyclone 'Keni'": [['cyclone', 'keni']], 
             "Storm 'Adrian'": [['storm','adrian']], 
             "Cyclone 'Kenneth'": [['cyclone', 'kenneth']], 
             "Cyclone 'Fani'": [['cyclone', 'fani']], 
             "Cyclone 'Wutip'": [['cyclone', 'wutip']], 
             'Typhoon Chaba (Chyba?)': [['typhoon','chaba'], ['typhoon', 'chyba']], 
             'Tropical storm Mindulle': [['storm', 'mindulle']], 
             'Tropical storm Marce (Tokage)': [['storm','marce'],['storm','tokage']], 
             "Cyclone 'Vardah'": [['cyclone','vardah']], 
             'Typhoon Lionrock': [['typhoon','lionrock']], 
             'Typhoon Malakas': [['typhoon','malakas']], 
             'Typhoon Helen (Megi)': [['typhoon','helen']], 
             'Measles': [['measles']], 'Lassa fever': [['lassa', 'fever']], 
             "Typhoon 'Nina' (Nock-Ten)": [['typhoon','nina'], ['typhoon','nock-ten']], 
             "Typhoon 'Lan'/'Paolo'": [['typhoon','lan'], ['typhoon','paolo']], 
             "Typhoon 'Talim'": [['typhoon','talim']], 
             "Cyclone 'Ockhi'": [['cyclone','ockhi']], 
             'Beatriz': [['beatriz']], 
             "Hurricane 'Franklin'": [['hurricane','franklin']], 
             "Typhoon 'Damrey' / 'Ramil'": [['typhoon','damrey'], ['typhoon','ramil']], 
             "Storm 'Tembin' (Vinta)": [['storm','tembin'],['storm','vinta']], 
             "Tropical storm 'Kai-Tak' (Urduja)": [['storm','kai-tak'], ['storm','urduja']], 
             "Cyclone 'Cempaka'": [['cyclone','cempaka']], 
             'Hurricane Katia': [['hurricane','katia']], 
             "Typhoon 'Doksuri'": [['typhoon','doksuri']], 
             "Cyclone 'Enawo'": [['cyclone','enawo']], 
             "Tropical storm 'Lidia'": [['storm','lidia']], 
             "Cyclone 'Dineo'": [['cyclone','dineo']], 
             'Mt. Agung': [['mt. agung'],['mount agung']], 
             "Typhoon 'Noru'": [['typhoon','noru']], 
             'Marburg': [['marburg']], 
             'Chikungunya': [['chikungunya']], 
             'Plague': [['plague']], 
             'Meningitis': [['meningitis']], 
             'Hepatitis E Virus': [['hepatitis e']], 
             'Tropical depression 02W (Crising)': [['depression','crising']], 
             'Influenza A': [['influenza a']], 
             "Cyclone 'Ava'": [['cyclone','ava']], 
             "Tropical storm 'Basyang' (Sanba)": [['storm','basyang'], ['storm','sanba']], 
             "Tropical depression 'Usman'": [['depression','usman']], 
             "Tropical storm 'Titli'": [['storm','titli']], 
             "Typhoon 'Yutu' (Rosita)": [['typhoon', 'yutu'], ['typhoon', 'rosita']], 
             'Anak Krakatoa': [['anak'], ['krakatoa']], 
             'Mount Etna': [['mount', 'etna'], ['etna']], 
             'Typhoon Jebi': [['typhoon', 'jebi']], 
             'Typhoon Trami': [['typhoon', 'trami']], 
             "Tyhoon 'Son Tinh'": [['typhoon', 'son', 'tinh']], 
             "Tropical storm 'Eliakim'": [['storm', 'eliakim']], 
             "Tropical storm 'Vicente'": [['storm', 'vicente']], 
             "Tropical storm 'Berguitta'": [['storm', 'berguitta']], 
             "Cyclone 'Mekunu'": [['cyclone', 'mekunu']], 
             "Cyclone 'Luban'": [['cyclone', 'luban']], 
             "Cyclone 'Gaja'": [['cyclone', 'gaja']], 
             "Tropical storm 'Phethai'": [['storm', 'pethai']], 
             "Typhoon 'Kong-Rey'": [['typhoon', 'kong-rey']], 
             'Hurricane Willa': [['hurricane', 'willa']], 
             "Tropical cyclone 'Penny'": [['cyclone', 'penny']], 
             'Mt. Mayon': [['mt.', 'mayon'], ['mount', 'mayon']], 
             'Agaton (01W)': [['agaton']], 
             "Storm 'Leslie'": [['storm', 'leslie']], 
             "Storm 'Norma'": [['storm', 'norma']], 
             "Cyclone 'Idai'": [['cyclone', 'idai']], 
             'Acute Encephalitis Syndrome (AES)': [['acute', 'encephalitis', 'syndrome']], 
             "Tropical depression 'Amang' (01W)": [['depression', 'amang']], 
             'Mount Ulawun and Manam': [['mount', 'ulawun', 'manam'], ['mount', 'ulawun'], ['mount', 'manam']], 
             'Hurricane Hermine': [['hurricane', 'hermine']], 
             'Tropical storm Aere': [['storm', 'aere']], 
             'Hurricane Harvey': [['hurricane', 'harvey']], 
             'Acute Watery Diarrhoea': [['acute', 'watery', 'diarrhea'], ['acute', 'watery', 'diarrhoea']], 
             'Wall Fire, Alamo Fire, Whittier Fire': [['wall', 'fire'], ['Alamo', 'fire'], ['Whittier', 'fire']], 
             'Tubbs, Atlas, Nuns Fires': [['tubes', 'atlas', 'nuns', 'fires'], ['tubes', 'nuns', 'fires'], ['nuns', 'fires'], ['tubes', 'fires']], 
             "'Thomas'": [['thomas']], 
             'Mt Monaro': [['mt', 'monaro'], ['mount', 'monaro']], 
             "Tropical cyclone 'Donna'": [['cyclone', 'donna']], 
             'Knysna fire': [['knysna', 'fire']], 'Kilauea': [['kilauea']], 
             'Ambryn': [['ambryn']], 
             'Carr and Mendocino Complex fires': [['carr', 'mendocino', 'fires']], 
             'Cyclone Gita': [['cyclone', 'gita']], 'Hurricane Florence': [['hurricane', 'florence']], 
             'Camp Fire': [['camp', 'fire']], 'Woolsey Fire': [['woolsey', 'fire']], 
             "Tropical depression 'Toraji'": [['depression', 'toraji']], 
             'Ebola': [['ebola']], 
             'Typhoid fever': [['typhoid', 'fever']], 
             'Currowan': [['currowan']], 
             'Tropical cyclone Lekima (Hanna)': [['cyclone', 'lekima'], ['cyclone', 'hanna']], 
             'FL-2019-000131': [['fl-2019-000131']], 
             "Tropical cylone 'Pabuk'": [['cyclone', 'pabuk']], 
             "Tropical cyclone 'Bulbul'": [['cyclone', 'bulbul']], 
             "Tropical cyclone 'Dorian'": [['cyclone', 'dorian']], 
             'Ubinas': [['ubinas']], 
             "Tropical cyclone 'Lingling'": [['cyclone', 'lingling']], 
             "Tropical cyclone 'Mitag'": [['cyclone', 'mitag']], 
             "Cyclone 'Belna'": [['cyclone', 'belna']], 
             "Storm 'Ciara' (Sabine)": [['storm', 'ciara'], ['storm', 'sabine']], 
             "Storm 'Amelie'": [['storm', 'amelie']], 
             "Tropical cylone 'Hagibis'": [['cyclone', 'hagibis']], 
             "Tropical cyclone 'Lorena'": [['cyclone', 'lorena']], 
             "Tropical storm 'Fernand'": [['storm', 'fernand']], 
             'Elsa, Fabien': [['elsa'], ['fabien']], 
             "Tropical cyclone 'Sarai'": [['cyclone', 'sarai']], 
             "Tropical storm 'Tapah'": [['storm', 'tapah']], 
             "Typhoon 'Faxai'": [['typhoon', 'faxai']], 
             "Tropical storm 'Narda'": [['storm', 'narda']], 
             'Whakaari/White Island volcano': [['whakaari', 'volcano'], ['white', 'island', 'volcano']], 
             "Cyclone 'Herold'": [['cyclone', 'herold']], 
             "'Gloria'": [['gloria']], 
             "Cyclone 'Tino'": [['cyclone', 'tino']], 
             "Tropical cyclone 'Kalmaegi' (Ramon)": [['cyclone', 'kalmaegi'], ['cyclone', 'ramon']], 
             "Tropical cyclone 'Phanfone' (Ursula)": [['cyclone', 'phanfone'], ['cyclone', 'ursula']], 
             "Tropical Storm 'Pawan'": [['storm', 'pawan']], 
             "Typhoon 'Matmo'": [['typhoon', 'matmo']], 
             "Tropical storm ' Imelda'": [['storm', 'imelda']], 
             "Tropical cyclone 'Nakri'": [['cyclone', 'nakri']], 
             "Tropical cyclone 'Danas'": [['cyclone', 'danas']], 
             "Tropical cyclone 'Podul'": [['cyclone', 'podul']], 
             "Tropical cyclone 'Kammuri' (Tisoy)": [['cyclone', 'kammuri'], ['cyclone', 'tisoy']], 
             "Tropical cylone 'Barry'": [['cyclone', 'barry']], 
             'Saddleridge fire & Sandalwood fire': [['saddleridge', 'fire'], ['sandalwood', 'fire']], 
             'Kincade Fire': [['kincade', 'fire']], 'Taal volcano': [['taal', 'volcano']]}

In [59]:
# from rdflib import Graph, URIRef, Literal, BNode
# from rdflib.namespace import FOAF, RDF

# g = Graph()
# g.bind("foaf", FOAF)

# bob = URIRef("http://example.org/people/Bob")
# linda = BNode()  # a GUID is generated

# name = Literal("Bob")
# age = Literal(24)

# g.add((bob, RDF.type, FOAF.Person))
# g.add((bob, FOAF.name, name))
# g.add((bob, FOAF.age, age))
# g.add((bob, FOAF.knows, linda))
# g.add((linda, RDF.type, FOAF.Person))
# g.add((linda, FOAF.name, Literal("Linda")))

# print(g.serialize())

for i in trange(len(disaster_info)):
    start_date, end_date = disaster_info.iloc[i]['Start Date'], disaster_info.iloc[i]['End Date']
    start_date_adj = start_date - timedelta(days=pre_margin)
    end_date_adj = end_date + timedelta(days=post_margin)
    candidate_articles = news_data[(news_data['Date']  < end_date_adj) & (news_data['Date']  > start_date_adj)]
    disaster_types = [disaster_info.iloc[i]['Disaster Type'], disaster_info.iloc[i]['Disaster Subtype'], disaster_info.iloc[i]['Disaster Subsubtype']]
    
    count_1 = 0
    count_2 = 0
    count_3 = 0
    for j, candidate in candidate_articles.iterrows():
        contains = False
        # _________________________________Mentions 1____________________________________________________
        # If event name is mentioned in an article it assumed to talk about the subject.
        if type(disaster_info.iloc[i]['Event Name']) == str:
            kws_1 = conv_dict[disaster_info.iloc[i]['Event Name']]
            for kws in kws_1:
                if all(x in candidate.Article.lower() for x in kws):
                    count_1 += 1
#                     print(candidate.Article)
                    contains = True
                    break
            if contains:
                continue
        
        # _________________________________Mentions 2____________________________________________________
        # If any of the locations in the locations list is mentioned together with the type of disaster or 
        # subtype or disaster, the article is counted towards mentions 2.
        if any(location.lower() in candidate.Article.lower() for location in disaster_info.iloc[i]['Location List'] if type(location) == str) and any(dis_type.lower() in candidate.Article.lower() for dis_type in disaster_types if type(dis_type) == str):
            count_2 += 1
            continue
    
    
        # _________________________________Mentions 3____________________________________________________
        # If the country is mentioned together with the type of disaster or 
        # subtype or disaster, the article is counted towards mentions 3.
        if type(disaster_info.iloc[i]['Country'].lower()) == str:
            if disaster_info.iloc[i]['Country'].lower() in candidate.Article.lower() and any(dis_type.lower() in candidate.Article.lower() for dis_type in disaster_types if type(dis_type) == str):
                    count_3 += 1
    
    if count_1+count_2+count_3!=0:
        print(f'c1: {count_1}, c2: {count_2}, c3: {count_3}')
        print(disaster_info.iloc[i]['Event Name'], disaster_info.iloc[i]['Start Date'])
        print()
    
    disaster_info.loc[i, 'mentions_1'] = count_1
    disaster_info.loc[i, 'mentions_2'] = count_2
    disaster_info.loc[i, 'mentions_3'] = count_3

  0%|                                      | 1/1517 [03:13<81:34:25, 193.71s/it]

c1: 4, c2: 0, c3: 0
Cyclone Roanu 2016-05-21 00:00:00



  0%|                                       | 2/1517 [03:22<35:41:00, 84.79s/it]

c1: 0, c2: 5, c3: 71
nan 2016-04-16 00:00:00



  0%|                                       | 3/1517 [04:50<36:21:28, 86.45s/it]

...

 94%|██████████████████████████████████▉  | 1432/1517 [6:36:21<12:15,  8.65s/it]

c1: 0, c2: 0, c3: 8
nan 2020-02-07 00:00:00



 95%|███████████████████████████████████  | 1435/1517 [6:36:54<13:37,  9.97s/it]

c1: 0, c2: 0, c3: 1
nan 2020-03-20 00:00:00



100%|█████████████████████████████████████| 1517/1517 [6:37:04<00:00, 15.71s/it]


In [205]:
disaster_info.groupby(['Event Name'])['Dis No'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(25)

,Event Name,count
13,Cholera,21
52,Hurricane 'Irma',16
40,Dengue,15
53,Hurricane 'Maria',9
82,Storm 'Ciara' (Sabine),9
54,Hurricane 'Matthew',8
116,Tropical depression'16/Hurricane 'Nate',8
69,Measles,6
43,Eleanor,5
0,'Friederike' (David),4


In [32]:
disaster_info.iloc[:30]

,Dis No,Year,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,Appeal,Declaration,AID Contribution ('000 US$),Dis Mag Value,Dis Mag Scale,Latitude,Longitude,River Basin,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,"Reconstruction Costs, Adjusted ('000 US$)","Insured Damages, Adjusted ('000 US$)","Total Damages, Adjusted ('000 US$)",Geo Locations,Start Date,End Date,Location List,mentions_1,mentions_2,mentions_3
0,2016-0175-BGD,2016,TC-2016-000052,Natural,Meteorological,Storm,Tropical cyclone,NaN,Cyclone Roanu,Bangladesh,BGD,Southern Asia,Asia,"Barisal; Noakhali, Lakshmipur, Chandpur (Chitt...",NaN,Flood,"Slide (land, mud, snow, rock)",NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,28.0,NaN,1083855.0,119700.0,1203555.0,NaN,NaN,731615.0,"Barguna, Bhola, Chandpur, Cox's Bazar, Lakshmi...",2016-05-21,2016-05-21,"[Barisal, Noakhali, Lakshmipur, Chandpur, Chit...",0,0,0
1,2016-0117-ECU,2016,EQ-2016-000035,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Ecuador,ECU,South America,Americas,"Guayaquil district (Guayas province), Muisne d...",NaN,NaN,"Slide (land, mud, snow, rock)",NaN,Yes,44992.0,8.0,Richter,0.382,-79.920,NaN,672.0,6274.0,383090.0,NaN,389364.0,4077535.0,682841.0,2438717.0,"Los Rios, Santa Elena, Santo Domingo de los Ts...",2016-04-16,2016-04-16,"[Guayaquil district, Guayas province, Muisne d...",0,0,0
2,2016-0095-CHN,2016,NaN,Natural,Hydrological,Flood,Flash flood,NaN,NaN,China,CHN,Eastern Asia,Asia,"Jiangxi Sheng, Hunan Sheng, Guangdong Sheng, G...",Monsoonal rain,"Slide (land, mud, snow, rock)",Hail,NaN,NaN,NaN,383224.0,Km2,24.704,113.991,NaN,5.0,NaN,216000.0,3300.0,219300.0,NaN,NaN,207291.0,"Guangdong Sheng, Guangxi Zhuangzu Zizhiqu, Gui...",2016-03-19,2016-03-22,"[Jiangxi Sheng, Hunan Sheng, Guangdong Sheng, ...",0,0,0
3,2016-0137-CHN,2016,NaN,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,China,CHN,Eastern Asia,Asia,"Anhui, Hubei, Fujian, Zhejiang, Jiangxi, Hunan...",Heavy rains,"Slide (land, mud, snow, rock)",Storm,NaN,NaN,NaN,NaN,Km2,NaN,NaN,Yangtze river,20.0,NaN,48000.0,NaN,48000.0,NaN,NaN,118278.0,"Anhui Sheng, Fujian Sheng, Guangdong Sheng, Gu...",2016-04-20,2016-04-28,"[Anhui, Hubei, Fujian, Zhejiang, Jiangxi, Huna...",0,0,0
4,2016-0041-FJI,2016,TC-2016-000014,Natural,Meteorological,Storm,Tropical cyclone,NaN,Cyclone Winston,Fiji,FJI,Melanesia,Oceania,"Savusavu (Cakaudrove district, Northern provin...",NaN,Flood,Surge,Yes,Yes,41862.0,325.0,Kph,NaN,NaN,NaN,45.0,144.0,540414.0,NaN,540558.0,NaN,60968.0,731615.0,"Ba, Bua, Cakaudrove, Lau, Lomaiviti, Macuata, ...",2016-02-20,2016-02-21,"[Savusavu, Cakaudrove district, Northern provi...",1,0,0
5,2016-0112-AFG,2016,FL-2016-000021,Natural,Hydrological,Flood,Flash flood,NaN,NaN,Afghanistan,AFG,Southern Asia,Asia,"Daykundi, Ghazni, Uruzgan provinces",NaN,"Slide (land, mud, snow, rock)",NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Daykundi, Ghazni, Uruzgan (Adm1).",2016-04-02,2016-04-08,"[Daykundi, Ghazni, Uruzgan provinces]",0,0,0
6,2016-0191-AFG,2016,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,Afghanistan,AFG,Southern Asia,Asia,"Badakhshan (Tashkan, Baharak, Darayem, Shahada...",NaN,"Slide (land, mud, snow, rock)",NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baharak, Darayem, Shuhada, Teshkan (Adm2).",2016-05-09,2016-05-12,"[Badakhshan, Tashkan, Baharak, Darayem, Shahad...",0,0,0
7,2016-0246-BOL,2016,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,Bolivia (Plurinational State of),BOL,South America,Americas,"Sur Yungas, Nor Yungas, Camacho, Bautista Saav...",Heavy rains,"Slide (land, mud, snow, rock)",NaN,NaN,NaN,NaN,190581.0,Km2,-12.843,-64.930,NaN,NaN,NaN,120.0,NaN,120.0,NaN,NaN,NaN,"Arani, Bautista Saavedra, Camacho, Carrasco, F...",2016-02-28,2016-03-03,"[Sur Yungas, Nor Yungas, Camacho, Bautista Saa...",0,0,0
8,2016-0077-BRA,2016,NaN,Natural,Hydrological,Flood,Flash flood,NaN,NaN,Bra